# Setup

## Imports

In [232]:
import numpy as np
import math
import cv2
import random
from tqdm import tqdm
import os
from vidgear.gears import WriteGear

## Consts

In [1]:
VIDEO_PATH = '../video_examples/SampleVideo_1280x720_1mb.mp4'

In [465]:
N_NON_PREDICTED_FRAMES = 4

## Helpers

In [394]:
def add_title(frame, title):
    black = [0,0,0]
    constant = cv2.copyMakeBorder(frame,10,10,10,10,cv2.BORDER_CONSTANT,value=black)
    
    violet= np.zeros((100, constant.shape[1], 3), np.uint8)
    violet[:] = (255, 0, 180)
    
    vcat = cv2.vconcat((violet, constant))

    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(vcat, title, (30,50), font, 2,(0,0,0), 3, 0)
    
    
    return vcat

# Process

## Pre Training

In [435]:
class ModelAPI():
    def __init__(self, model):
        self.model = model
    
    def fine_tune(self, original_video):
        kernel = np.ones((5,5),np.float32)/25
                
        def model(prev_frames):
#             return prev_frames[-1]
            return cv2.filter2D(prev_frames[-1],-1,kernel)
        
        self.model = model
    
    def predict(self, prev_frames):
        return self.model(prev_frames)

In [436]:
def load_model():
    def model(prev_frames):
        raise Exception('Model not fine tuned')
        
    return ModelAPI(model)

## Compression

In [438]:
model_api = load_model()

In [447]:
# Reading video
cap = cv2.VideoCapture(VIDEO_PATH)
frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Initializing frames
frames = {
    k: np.empty((frameCount, frameHeight, frameWidth, 3), np.dtype('uint8'))
    for k in ['original', 'original_diffs', 'original_prev', 'predicted',  'predicted_diffs', 'filler']
}

# Printing video shape
display(frames['original'].shape)

for frame_i in tqdm(range(frameCount), total=frameCount):
    # Reading next frame
    ret,frames['original'][frame_i] = cap.read()
    if not ret:
        break

cap.release()

(132, 720, 1280, 3)

100%|██████████| 132/132 [00:00<00:00, 423.42it/s]


In [448]:
model_api.fine_tune(frames['original'])

In [457]:
for frame_i in tqdm(range(N_NON_PREDICTED_FRAMES, frameCount), total=frameCount-N_NON_PREDICTED_FRAMES):
#     if frame_i > 10: break
    
    next_frame = frames['original'][frame_i]
    prev_frame = frames['original'][frame_i-N_NON_PREDICTED_FRAMES]
    predicted_next_frame = model_api.predict(frames['original'][:frame_i])
    
    frames['original_prev'][frame_i] = prev_frame
    frames['predicted'][frame_i] = predicted_next_frame
    
    frames['original_diffs'][frame_i] = next_frame - prev_frame
    frames['predicted_diffs'][frame_i] = next_frame - predicted_next_frame

    


100%|██████████| 128/128 [00:00<00:00, 130.15it/s]


### Show

In [451]:
COMP_WINDOW_NAME = 'Compression Demo'
VID_PER_ROW = 3


cv2.namedWindow(COMP_WINDOW_NAME)

for frame_i in range(frameCount):
    if frame_i > 10: break
    v =[]
    keys = list(frames.keys())
    
    for k_i in range(math.ceil(len(keys)/VID_PER_ROW)):
        
        v.append(
            cv2.hconcat([
                add_title(
                        frames[keys[k_i*VID_PER_ROW + i]][frame_i],
                        f"{keys[k_i*VID_PER_ROW + i]} ({frame_i})"
                    )
                for i in range(VID_PER_ROW)
            ])        
        )
        
    p = cv2.vconcat(v)
    cv2.imshow(COMP_WINDOW_NAME, p)

    cv2.waitKey(0)

cv2.destroyWindow(COMP_WINDOW_NAME)

cv2.waitKey(1)

-1

## Decompression

In [452]:
calculated_frames = np.empty((frameCount, frameHeight, frameWidth, 3), np.dtype('uint8'))

In [453]:
for frame_i in tqdm(range(0, N_NON_PREDICTED_FRAMES), total=N_NON_PREDICTED_FRAMES): 
    calculated_frames[frame_i] = frames['original'][frame_i]

100%|██████████| 4/4 [00:00<00:00, 489.53it/s]


In [456]:
for frame_i in tqdm(range(N_NON_PREDICTED_FRAMES, frameCount), total=frameCount-N_NON_PREDICTED_FRAMES):
#     if frame_i > 10: break
    
    next_frame = frames['original'][frame_i]
    prev_frame = frames['original'][frame_i-N_NON_PREDICTED_FRAMES]
    
    predicted_next_frame = model_api.predict(calculated_frames[:frame_i])
    calculated_frames[frame_i] = predicted_next_frame + frames['predicted_diffs'][frame_i]



100%|██████████| 128/128 [00:00<00:00, 159.24it/s]


Asserting that the compression worked

In [464]:
assert (calculated_frames == frames['original']).all()

# Evaluation

In [485]:
compress_frames_keys = ['original', 'predicted_diffs', 'original_diffs']

In [486]:
compress_frames = {
    k:frames[k]
    for k in compress_frames_keys
}

For more information about params https://trac.ffmpeg.org/wiki/Encode/H.264#Howdothedifferentpresetsinfluenceencodingtime

In [505]:
output_params = {"-vcodec":"libx264", "-crf": 0, "-preset": "veryslow"} #define (Codec,CRF,preset) FFmpeg tweak parameters for writer

In [506]:
compress_frames_writers = {
    k: WriteGear(
        output_filename = f'{k}.mp4',
        compression_mode = True,
        logging = True,
        **output_params
    )
    for k in compress_frames_keys
}

18:27:48 :: WriteGear :: DEBUG :: Compression Mode is enabled therefore checking for valid FFmpeg executables.
18:27:48 :: WriteGear :: DEBUG :: Output Parameters: {'-vcodec': 'libx264', '-crf': 0, '-preset': 'veryslow'}
18:27:48 :: Helper :: DEBUG :: Final FFmpeg Path: ffmpeg
18:27:48 :: Helper :: DEBUG :: FFmpeg validity Test Passed!
18:27:48 :: Helper :: DEBUG :: Found valid FFmpeg Version: `b'4.4'` installed on this system
18:27:48 :: WriteGear :: DEBUG :: Found valid FFmpeg executables: `ffmpeg`.
18:27:48 :: WriteGear :: DEBUG :: Compression Mode is configured properly!
18:27:48 :: WriteGear :: DEBUG :: Compression Mode is enabled therefore checking for valid FFmpeg executables.
18:27:48 :: WriteGear :: DEBUG :: Output Parameters: {'-vcodec': 'libx264', '-crf': 0, '-preset': 'veryslow'}
18:27:48 :: Helper :: DEBUG :: Final FFmpeg Path: ffmpeg
18:27:48 :: Helper :: DEBUG :: FFmpeg validity Test Passed!
18:27:48 :: Helper :: DEBUG :: Found valid FFmpeg Version: `b'4.4'` installed on

In [507]:
for frame_i in tqdm(range(N_NON_PREDICTED_FRAMES, frameCount), total=frameCount-N_NON_PREDICTED_FRAMES):
    for k in compress_frames_keys:
        compress_frames_writers[k].write(compress_frames[k][frame_i])
        
for writer in compress_frames_writers.values():
    writer.close()

  0%|          | 0/128 [00:00<?, ?it/s]18:27:49 :: WriteGear :: DEBUG :: InputFrame => Height:720 Width:1280 Channels:3
18:27:49 :: WriteGear :: DEBUG :: Executing FFmpeg command: `ffmpeg -y -f rawvideo -vcodec rawvideo -s 1280x720 -pix_fmt bgr24 -i - -vcodec libx264 -crf 0 -preset veryslow /Users/nirendy/school-repo/nn-based-video-compression/notebooks/original.mp4`
18:27:49 :: WriteGear :: DEBUG :: InputFrame => Height:720 Width:1280 Channels:3
18:27:49 :: WriteGear :: DEBUG :: Executing FFmpeg command: `ffmpeg -y -f rawvideo -vcodec rawvideo -s 1280x720 -pix_fmt bgr24 -i - -vcodec libx264 -crf 0 -preset veryslow /Users/nirendy/school-repo/nn-based-video-compression/notebooks/predicted_diffs.mp4`
18:27:49 :: WriteGear :: DEBUG :: InputFrame => Height:720 Width:1280 Channels:3
18:27:49 :: WriteGear :: DEBUG :: Executing FFmpeg command: `ffmpeg -y -f rawvideo -vcodec rawvideo -s 1280x720 -pix_fmt bgr24 -i - -vcodec libx264 -crf 0 -preset veryslow /Users/nirendy/school-repo/nn-based-vid

{"-vcodec":"libx264", "-crf": 0, "-preset": "veryslow"}

In [508]:
for k in compress_frames_keys + ['source']:
    if k=='source':
        size = os.path.getsize(f'../video_examples/SampleVideo_1280x720_1mb.mp4')
    else:
        size = os.path.getsize(f'{k}.mp4')
        
    size = size // 10**6
    print(f"Size of file '{k}' is {size} Mb")
    

Size of file 'original' is 34 Mb
Size of file 'predicted_diffs' is 139 Mb
Size of file 'original_diffs' is 113 Mb
Size of file 'source' is 1 Mb


{"-vcodec":"libx264", "-crf": 0, "-preset": "fast"}

In [504]:
for k in compress_frames_keys + ['source']:
    if k=='source':
        size = os.path.getsize(f'../video_examples/SampleVideo_1280x720_1mb.mp4')
    else:
        size = os.path.getsize(f'{k}.mp4')
        
    size = size // 10**6
    print(f"Size of file '{k}' is {size} Mb")
    

Size of file 'original' is 34 Mb
Size of file 'predicted_diffs' is 145 Mb
Size of file 'original_diffs' is 114 Mb
Size of file 'source' is 1 Mb


No compression

In [499]:
for k in compress_frames_keys + ['source']:
    if k=='source':
        size = os.path.getsize(f'../video_examples/SampleVideo_1280x720_1mb.mp4')
    else:
        size = os.path.getsize(f'{k}.mp4')
        
    size = size // 10**6
    print(f"Size of file '{k}' is {size} Mb")
    

Size of file 'original' is 15 Mb
Size of file 'predicted_diffs' is 95 Mb
Size of file 'original_diffs' is 74 Mb
Size of file 'source' is 1 Mb
